In [48]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import pickle

In [55]:
# import pickle file from ./test_policy/test_trajectory.pkl
with open('./test_policy/test_fully-connected_10/test_trajectory.pkl', 'rb') as f:
    trajectories = pickle.load(f)

print(len(trajectories))

# reshape this into a 1000 by 70 by 17 array
# test_trajectory = test_trajectory.reshape(1000, 70, 17)



300
